In [1]:
orig_metadata_fname = "data\\augmented-metadata_wide_tic.tsv"
update_metadata_fname = "data\\UW_UW-Phase-2a-Performer-Submission_1.0_UW-UW-Phase-2a-Performer-Submission-SUB1218_sample-level.csv"
output_metadata_fname = "data\\unblinded-augmented-metadata_wide_tic.tsv"

In [2]:
import pandas as pd

orig_metadata = pd.read_csv(orig_metadata_fname, sep='\t', na_values=['na'])
update_metadata = pd.read_csv(update_metadata_fname, na_values=[''])

print('orig metadata shape', orig_metadata.shape)
print('update metadata shape', update_metadata.shape)

orig metadata shape (288, 24)
update metadata shape (576, 23)


In [3]:
def merge_metadata(orig_metadata: pd.DataFrame, update_metadata: pd.DataFrame) -> pd.DataFrame:
    # Get the columns that will be merged from the update_metadata
    update_cols = ['sample', 'Is Irradiated?', 'Radiation Dose(cGy)', 'daysaftertreatment', 'Dose Rate']
    
    # Ensure we only use columns that actually exist in the update_metadata
    available_update_cols = [col for col in update_cols if col in update_metadata.columns]
    
    # Perform a left join to keep all rows from orig_metadata
    merged_df = orig_metadata.merge(
        update_metadata[available_update_cols],
        left_on='TRA_number', 
        right_on='sample', 
        how='left'
        # No suffixes specified, so pandas will use default "_x" and "_y"
    )
    
    # Get columns that ended up with "_x" and "_y" suffixes
    x_cols = [col for col in merged_df.columns if col.endswith('_x')]
    base_cols = [col[:-2] for col in x_cols]  # Remove the "_x" suffix
    
    # Handle potentially overlapping columns
    for base_col in base_cols:
        x_col = f"{base_col}_x"
        y_col = f"{base_col}_y"
        
        # Special handling for daysaftertreatment column
        if base_col == 'daysaftertreatment':
            # Create temporary clean columns for comparison
            x_clean = merged_df[x_col].copy()
            y_clean = merged_df[y_col].copy()
            
            # Clean y column - remove " days" suffix if it exists and convert to numeric
            y_clean = y_clean.astype(str).str.replace(' days', '', regex=False).replace('', float('nan')).astype(float)
            
            # Convert x column to numeric if it's not already
            if pd.api.types.is_numeric_dtype(x_clean):
                x_clean = x_clean.astype(float)
            else:
                x_clean = x_clean.astype(str).str.replace(' days', '', regex=False).replace('', float('nan')).astype(float)
            
            # Check if values match where both exist and are not null
            mask = ~x_clean.isna() & ~y_clean.isna()
            if mask.any() and not (x_clean[mask] == y_clean[mask]).all():
                mismatched = pd.DataFrame({
                    x_col: merged_df.loc[mask, x_col],
                    y_col: merged_df.loc[mask, y_col],
                    'x_clean': x_clean[mask],
                    'y_clean': y_clean[mask]
                })
                mismatched = mismatched[x_clean[mask] != y_clean[mask]]
                raise ValueError(f"Mismatched values found in column '{base_col}':\n{mismatched}")
            
            # Create merged column as integer where possible
            merged_df[base_col] = y_clean.fillna(x_clean).astype('Int64')  # Use nullable integer type
            
        else:
            # Standard handling for other columns
            # Check if values match where both exist and are not null
            mask = ~merged_df[x_col].isna() & ~merged_df[y_col].isna()
            if mask.any() and not (merged_df.loc[mask, x_col] == merged_df.loc[mask, y_col]).all():
                mismatched = merged_df.loc[mask & (merged_df[x_col] != merged_df[y_col])]
                raise ValueError(f"Mismatched values found in column '{base_col}':\n{mismatched[[x_col, y_col]]}")
            
            # Create merged column - take update value (y) if available, otherwise keep original (x)
            merged_df[base_col] = merged_df[y_col].fillna(merged_df[x_col])
        
        # Drop the duplicate columns
        merged_df = merged_df.drop(columns=[x_col, y_col])
    
    # Drop the redundant 'sample' column from update_metadata
    if 'sample' in merged_df.columns:
        merged_df = merged_df.drop(columns=['sample'])
    
    return merged_df

In [4]:
augmented_metadata = merge_metadata(orig_metadata, update_metadata)
print('augmented_metadata shape', augmented_metadata.shape)


augmented_metadata shape (288, 26)


In [5]:
augmented_metadata.to_csv(output_metadata_fname, sep="\t", index=False)